In [ ]:
library('MASS')

In [ ]:
N = 250
P = 500

In [ ]:
K = floor(P/3)
alpha = array(rnorm(K*P),c(K,P))
W = array(rnorm(N*K),c(N,K))
W_test = array(rnorm(N*10*K),c(N*10,K))

In [ ]:
X = W%*%alpha

In [ ]:
X_test = W_test%*%alpha

In [ ]:
dim(X_test)

In [ ]:
beta = rnorm(P)
beta[50:500] <- 0 

In [ ]:
plot(beta)

In [ ]:
sig = P*.05

In [ ]:
etas = as.vector(X%*%beta)
head(etas)

$$Y \mid X \sim Bern(logistic(X\beta))$$

In [ ]:
?rbinom

In [ ]:
y = sapply(etas,function(eta)rbinom(n=1,size=1,prob=1/(1+exp(-eta))))
y

In [ ]:
df_train = data.frame(X)
df_train$y <- y
head(df_train)

In [ ]:
plot(X%*%beta,y)

In [ ]:
plot(X%*%beta,1/(1+exp(-X%*%beta)))

In [ ]:
etas_test = as.vector(X_test%*%beta)
y_test = sapply(etas_test,function(eta)rbinom(n=1,size=1,prob=1/(1+exp(-eta))))
df_test = data.frame(X_test)
df_test$y <- y_test

In [ ]:
naive_fit = glm(y~.,data=df_train,family='binomial')

In [ ]:
naive_fit$coefficients

In [ ]:
lambda = exp(seq(-50,20,length.out=100))
lambda

In [ ]:
library('glmnet')

In [ ]:
fit.ridge = glmnet(x=X,y=y,family="binomial",alpha=0,lambda=lambda)

In [ ]:
dim(fit.ridge$beta)

In [ ]:
matplot(log(fit.ridge$lambda),t(fit.ridge$beta),type='l',xlab="log(lam)",ylab="beta")

In [ ]:
cv.ridge = cv.glmnet(x=X,y=y,family="binomial",alpha=0,lambda=lambda)

In [ ]:
plot(cv.ridge)

In [ ]:
lam_hat = cv.ridge$lambda.min
log(lam_hat)

In [ ]:
lam_hat = cv.ridge$lambda.1se
log(lam_hat)

In [ ]:
best.ridge = glmnet(x=X,y=y,family="binomial",alpha=0,lambda=lam_hat)

In [ ]:
sample(best.ridge$beta,20)

In [ ]:
plot(beta,as.vector(best.ridge$beta))

In [ ]:
p_hat = unlist(predict(best.ridge,newx=X_test,type='response'))

In [ ]:
plot(X_test%*%best.ridge$beta,p_hat)

In [ ]:
plot(X_test%*%beta,p_hat)

In [ ]:
y_hat = as.factor(as.vector(predict(best.ridge,newx=X_test,type='class')))

In [ ]:
head(y_hat)

In [ ]:
y_test_f = as.factor(y_test)
head(y_test_f)

In [ ]:
library('caret')

In [ ]:
cm = caret::confusionMatrix(data=y_hat,reference=y_test_f)

In [ ]:
cm

### lasso 

In [ ]:
fit.lasso = glmnet(x=X,y=y,family="binomial",alpha=1,lambda=lambda)

In [ ]:
matplot(log(fit.lasso$lambda),t(fit.lasso$beta),type='l',xlab="log(lam)",ylab="beta")

In [ ]:
cv.lasso = cv.glmnet(x=X,y=y,family="binomial",alpha=1,lambda=lambda)

In [ ]:
plot(cv.lasso)

In [ ]:
lam_hat = cv.lasso$lambda.min
log(lam_hat)

In [ ]:
lam_hat = cv.lasso$lambda.1se
log(lam_hat)

In [ ]:
best.lasso = glmnet(x=X,y=y,family="binomial",alpha=1,lambda=lam_hat)

In [ ]:
plot(beta,as.vector(best.lasso$beta))

In [ ]:
y_hat = as.factor(as.vector(predict(best.lasso,newx=X_test,type='class')))
cm = caret::confusionMatrix(data=y_hat,reference=y_test_f)
cm

### elastic net

In [ ]:
lam_seq = exp(seq(-10,10,length.out=100))
fit.en = glmnet(x=X,y=y,family="binomial",alpha=1/2,lambda=lam_seq)
fit.en.cv = cv.glmnet(x=X,y=y,family="binomial",alpha=1/2,lambda=lam_seq)

In [ ]:
matplot(log(fit.en$lambda),t(fit.en$beta),type='l',xlab="log(lam)",ylab="beta")

In [ ]:
plot(fit.en.cv)

In [ ]:
fit.en = glmnet(x=X,y=y,family="binomial",alpha=1/2,lambda=fit.en.cv$lambda.1se)

In [ ]:
y_hat = as.factor(as.vector(predict(fit.en,newx=X_test,type='class')))
cm = caret::confusionMatrix(data=y_hat,reference=y_test_f)
cm